In [1]:
!pip install pydot
!pip install -U sagemaker

You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 225kB 48.6MB/s ta 0:00:01
Requirement not upgraded as not directly required: boto3>=1.9.213 in /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages (from sagemaker) (1.9.230)
Requirement not upgraded as not directly required: numpy>=1.9.0 in /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages (from sagemaker) (1.16.4)
Requirement not upgraded as not directly required: protobuf>=3.1 in /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages (from sagemaker) (3.7.1)
Requirement not upgraded as not directly required: scipy>=0.19.0 in /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages (from sagemaker) (1.3.0)
Requirement not upgraded as not directly required: urllib3<1.25,>=1.21 in /home/ec2-user/anaconda3/envs/tensorflow_p3

You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import keras
import sagemaker
from keras.models import load_model
import numpy as np
import os

import numpy as np
import matplotlib.pyplot as plt
import warnings
import tensorflow as tf

from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants
from keras.models import model_from_json
from sklearn.preprocessing import MinMaxScaler

from keras.layers import Input, Embedding, Flatten, Dot, Dense, LSTM, Activation
from keras.models import Model
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras import Sequential

warnings.filterwarnings('ignore')
%matplotlib inline
sess = sagemaker.Session()
role = sagemaker.get_execution_role()


Using TensorFlow backend.


### Copy the model artifact from S3 and get a visual representation 

In [11]:
# !aws s3 cp model-versions/1L-100%-model.h5 s3://strata-workshop/model-data/

upload: model-versions/1L-100%-model.h5 to s3://strata-workshop/model-data/1L-100%-model.h5


In [3]:
loaded_model = load_model('full-model-v3.h5')

W0924 11:18:58.331929 140191001786176 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0924 11:18:58.349981 140191001786176 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0924 11:18:58.353780 140191001786176 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0924 11:18:59.241332 140191001786176 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:13

In [4]:
from keras.utils import plot_model
plot_model(loaded_model, to_file='img-model.png')

![alt text](img-model.png "Base Model Architecture")

### Download the test data and run predictions against your endpoint

In [35]:
def read_data(f_name):
    
    df = pd.read_csv(f_name, sep='\t', nrows = 1000)
        
    df['docs_to_embed'] = df['product_title'] + ', ' + df['review_headline'] + ', ' + df['review_body']
    
    df.drop(['product_title', 'review_headline', 'review_body'], axis=1, inplace=True)
    
    # remove nans
    df.dropna(inplace=True)
    
#     df['customer_id'] = str(df['customer_id'])
 
    return df

df = read_data('micro_data.tsv')


In [36]:
def get_label(df):
    labels = [1 if int(x) >= 4 else 0 for x in df['star_rating'] ]
    return labels

def get_encoded_ids(df, id_name):

    assert id_name in ['product_id', 'docs_to_embed']
    
    if id_name == 'product_id':
        vocab_size = 4412
    
    elif id_name == 'docs_to_embed':
        vocab_size = 34467
    
    docs = df[id_name].values.tolist()
        
    encoded_ids = [one_hot(d, vocab_size) for d in docs]

    return np.array(encoded_ids)

def get_padded_documents(df):
    encoded_docs = get_encoded_ids(df, 'docs_to_embed')
    
    max_length = 2261
    
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
    
    return padded_docs


labels = get_label(df)


In [37]:
def get_model_input_specs(df):
    n_users = 9928
    n_products = 4412
    max_length = 2261
    vocab_size = 34467

    return vocab_size, max_length, n_users, n_products

def get_scaled(df, col_name):
    x = [int(x) for x in df[col_name]]
    
    x = np.reshape(x, (-1, 1))

    scaler_x = MinMaxScaler()

    scaler_x.fit(x)
    
    xscale = scaler_x.transform(x)

    return xscale

def get_model_input_data(df):
    
    padded_docs = get_padded_documents(df)
    
    encoded_product_ids = get_encoded_ids(df, 'product_id')

    votes = get_scaled(df, 'total_votes')
    
    return [padded_docs, votes, encoded_product_ids, df['customer_id']]

In [38]:
data_input = get_model_input_data(df)

In [30]:
y_pred = loaded_model.predict(data_input)

In [40]:
df['star_rating'].value_counts()

5    533
4    202
1    131
3     83
2     51
Name: star_rating, dtype: int64

In [33]:
len(y_pred)

1000

In [39]:
df.shape

(1000, 10)

### Now, retrain the model using  the data you just labeled

In [20]:
!aws s3 cp s3://strata-workshop/gt_sample/gt_label_sample_output.csv .

download: s3://strata-workshop/gt_sample/gt_label_sample_output.csv to ./gt_label_sample_output.csv


In [21]:
for layer in loaded_model.layers[:-1]:
    layer.trainable=False

for layer in loaded_model.layers[-2:]:
    layer.trainable=True

In [23]:
# get the new data from the labeling job

df = pd.read_csv('gt_label_sample_output.csv', sep='\t', names=['customer_id', 'product_id', 'product_title', 'product_category', 'star_rating', 'total_votes', 'review_headline', 'review_body'])

df['docs_to_embed'] = df['product_title'] + ', ' + df['review_headline'] + ', ' + df['review_body']

df.drop(['product_title', 'review_headline', 'review_body'], axis=1, inplace=True)

# remove nans
df.dropna(inplace=True)

In [24]:
new_data_input  = get_model_input_data(df)

In [25]:
new_labels = get_label(df)

In [27]:
history = loaded_model.fit(new_data_input, new_labels, epochs=1, verbose=1, validation_split=0.2)  

loaded_model.save('/home/ec2-user/SageMaker/transfer-model.h5')

# write to json 
json_string = loaded_model.to_json()
with open("transfer-model.json", "w") as json_file:
    json_file.write(json_string)

# save weights
loaded_model.save_weights('transfer-model.h5')

Train on 22 samples, validate on 6 samples
Epoch 1/1
22/22 [==============================] - 8s 355ms/step - loss: 3.5413 - val_loss: 1.0010


### Now use the test data against the new model you just trained using your labeled data

In [28]:
y_pred = loaded_model.predict(data_input)

In [64]:
y_pred

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)